# Segmenting and Clustering Neighborhoods in Toronto

## Part 1

Finding Toronto postal codes for neighborhoods

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
scrap_url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
req = requests.get(scrap_url)

In [4]:
soup = BeautifulSoup(req.text, 'html5lib')

Parsing url data into dataframe

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df.shape

(103, 3)

---

## Part 2

In [7]:
!pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [8]:
import geocoder

Code below dont work because of geocodder package, as said in assignment - it is very unreliable.

In [ ]:
latitude = []
longitude = []
for postal_code in df['PostalCode']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google(f'{postal_code}, Toronto, Ontario')
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
df['Latitude'] = latitude
df['Longitude'] = longitude

We are using CSV instead

In [9]:
df_lat_lng = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [10]:
df_lat_lng.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Merging two dataframes into one

In [11]:
df = pd.merge(df, df_lat_lng, on='PostalCode')

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


---

## Part 3

In [13]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [14]:
import folium
from geopy.geocoders import Nominatim

Print how many unique boroughs and neighborhoods we have.

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


Find coordinates of Toronto

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Show all unique boroughs

In [18]:
for borough in df['Borough'].unique():
    print(borough)

North York
Downtown Toronto
Queen's Park
Etobicoke
Scarborough
East York
York
East Toronto
West Toronto
East York/East Toronto
Central Toronto
Mississauga
Downtown Toronto Stn A
Etobicoke Northwest
East Toronto Business


Assignment offer us to analyze everything which have 'Totonto' in borough name, but I choise to analyze everything with 'York'.

Find list with all 'York' boroughs

In [19]:
york_borough = []
for borough in df['Borough'].unique():
    if 'york' in borough.lower():
        york_borough.append(borough)
york_borough

['North York', 'East York', 'York', 'East York/East Toronto']

Make new dataframe only with 'York' boroughs

In [20]:
york_data = df[df['Borough'].isin(york_borough)].reset_index(drop=True)
york_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


Let's visualize North Yorn and it's neighborhoods

In [21]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of North York are {latitude}, {longitude}.')

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [22]:
# create map of North York using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

Insert Foursquare credentials

In [23]:
CLIENT_ID = 'NOPE' # your Foursquare ID
CLIENT_SECRET = 'ALSO NOPE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Copy function from NY example
This function will fetch nearby venues by coordinates and given radius

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Gather all york venues into dataframe

In [25]:
york_venues = getNearbyVenues(names=york_data['Neighborhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale South
Downsview Northwest
Runnymede, The Junction North
Weston
York Mills West
Willowdale West


Check shape of york venues dataframe

In [26]:
print(york_venues.shape)
york_venues.head()

(321, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Let's check how many venues were returned for each neighborhood

In [27]:
york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Don Mills South,19,19,19,19,19,19
Downsview Central,3,3,3,3,3,3
Downsview East,3,3,3,3,3,3


Check how many unique categories we get

In [28]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 120 uniques categories.


Start analyzing each neighborhood

In [29]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
york_onehot.shape

(321, 121)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
print(york_grouped.shape)
york_grouped

(32, 121)


,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Don Mills South,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Downsview East,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.10
1            Bank  0.10
2     Bridal Shop  0.05
3  Ice Cream Shop  0.05
4     Gas Station  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2  Japanese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1          Restaurant  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.08
4    Sushi Restaurant  0.04


----Caledonia-Fairbanks----
               venue  freq
0               Park  0.50
1      Women's Store  0.25
2               Pool  0.25
3  Accessories Store  0.00
4  Korean Restaurant  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                venue  freq
0  Turkish Restaurant  0.25
1      Sandwich Place  0.25
2      Discount Store  0.25
3         Coffee Shop  0.

NY example function for sorting venues in dataframe

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Deli / Bodega,Restaurant,Sandwich Place,Ice Cream Shop,Pizza Place,Diner,Pharmacy
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Greek Restaurant,Fast Food Restaurant,Grocery Store,Hobby Shop,Indian Restaurant,Juice Bar
3,Caledonia-Fairbanks,Park,Pool,Women's Store,Furniture / Home Store,Fried Chicken Joint,Chocolate Shop,Clothing Store,Gas Station,Coffee Shop,Comfort Food Restaurant
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Coffee Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


#### Clustering

In [138]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Initialize cluster object and build clusters

In [139]:
# set number of clusters
kclusters = 3

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 2, 2, 0, 2], dtype=int32)

We pick kclusters=3 since it is giving us most ligitimate view of clusters.

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [140]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

york_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Financial or Legal Service,Yoga Studio,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Carpet Store,Vietnamese Restaurant,Dog Run,Comfort Food Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,2.0,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Baseball Field,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2.0,Pizza Place,Breakfast Spot,Intersection,Bank,Athletics & Sports,Flea Market,Gym / Fitness Center,Pharmacy,Gastropub,Construction & Landscaping


Check cluster labels for errors

In [141]:
york_merged['Cluster Labels'].unique

<bound method Series.unique of 0     0.0
1     2.0
2     2.0
3     2.0
4     2.0
5     2.0
6     2.0
7     2.0
8     2.0
9     0.0
10    2.0
11    2.0
12    2.0
13    2.0
14    2.0
15    2.0
16    0.0
17    2.0
18    0.0
19    0.0
20    0.0
21    0.0
22    1.0
23    NaN
24    2.0
25    2.0
26    2.0
27    2.0
28    2.0
29    2.0
30    2.0
31    NaN
32    0.0
33    2.0
Name: Cluster Labels, dtype: float64>

Cluster Labes have Nan label, drop it. Also column casted as float64 type.

In [142]:
york_merged = york_merged.dropna(subset=['Cluster Labels'])
york_merged = york_merged.astype({'Cluster Labels': int})
york_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Bus Stop,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Portuguese Restaurant,Hockey Arena,Financial or Legal Service,Yoga Studio,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Carpet Store,Vietnamese Restaurant,Dog Run,Comfort Food Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,2,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Baseball Field,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,Pizza Place,Breakfast Spot,Intersection,Bank,Athletics & Sports,Flea Market,Gym / Fitness Center,Pharmacy,Gastropub,Construction & Landscaping


Finally, let's visualize the resulting clusters

In [144]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

Cluster 0

In [146]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Bus Stop,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,York,0,Park,Pool,Women's Store,Furniture / Home Store,Fried Chicken Joint,Chocolate Shop,Clothing Store,Gas Station,Coffee Shop,Comfort Food Restaurant
16,East York/East Toronto,0,Park,Coffee Shop,Convenience Store,Intersection,Discount Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice
18,North York,0,Park,Airport,Business Service,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
19,North York,0,Park,Yoga Studio,Golf Course,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
20,North York,0,Hotel,Park,Shopping Mall,Bank,Yoga Studio,Dessert Shop,Department Store,Deli / Bodega,Curling Ice,Diner
21,North York,0,Park,Construction & Landscaping,Bakery,Basketball Court,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
32,North York,0,Park,Convenience Store,Yoga Studio,Golf Course,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice


Cluster 1

In [147]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,1,Pizza Place,Gym,Yoga Studio,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


Cluster 2

In [148]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Coffee Shop,Portuguese Restaurant,Hockey Arena,Financial or Legal Service,Yoga Studio,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,North York,2,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Carpet Store,Vietnamese Restaurant,Dog Run,Comfort Food Restaurant
3,North York,2,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Baseball Field,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
4,East York,2,Pizza Place,Breakfast Spot,Intersection,Bank,Athletics & Sports,Flea Market,Gym / Fitness Center,Pharmacy,Gastropub,Construction & Landscaping
5,North York,2,Park,Asian Restaurant,Sushi Restaurant,Japanese Restaurant,Metro Station,Bakery,Construction & Landscaping,Comfort Food Restaurant,Convenience Store,Coffee Shop
6,North York,2,Coffee Shop,Gym,Restaurant,Supermarket,Bike Shop,Grocery Store,Asian Restaurant,Chinese Restaurant,Italian Restaurant,Sandwich Place
7,East York,2,Skating Rink,Park,Beer Store,Curling Ice,Bus Stop,Video Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,York,2,Hockey Arena,Field,Trail,Cosmetics Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Curling Ice,Yoga Studio
10,East York,2,Coffee Shop,Bank,Furniture / Home Store,Sporting Goods Shop,Burger Joint,Beer Store,Sandwich Place,Bike Shop,Restaurant,Breakfast Spot
11,North York,2,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
